<a href="https://colab.research.google.com/github/sakshi2215/First_flaskblog/blob/master/mediapipeye_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 !pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 50.6 MB/s eta 0:00:00


In [23]:
import cv2 as cv
import mediapipe as mp
import numpy as np
from google.colab.patches import cv2_imshow
# Constants
FONTS = cv.FONT_HERSHEY_COMPLEX

# Face boundary indices
FACE_OVAL = [10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136, 172, 58, 132, 93, 234, 127, 162, 21, 54, 103, 67, 109]

# Left eye indices
LEFT_EYE = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398]

# Right eye indices
RIGHT_EYE = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246]

map_face_mesh = mp.solutions.face_mesh

# Read the image
image = cv.imread("/content/dreamy-happy-man-close-eyes-wait-surprise.jpg")

if image is None:
    print("Failed to load image.")
else:
    with map_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
        # Convert image from RGB to BGR
        rgb_image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

        results = face_mesh.process(rgb_image)
        if results.multi_face_landmarks:
            mesh_coords = [(int(point.x * image.shape[1]), int(point.y * image.shape[0])) for point in results.multi_face_landmarks[0].landmark]

            # Create separate images for the left and right eyes
            left_eye_image = np.zeros_like(image)  # Initialize with black image
            right_eye_image = np.zeros_like(image)  # Initialize with black image

            # Extract the bounding rectangle for the left eye
            left_eye_rect = cv.boundingRect(np.array([mesh_coords[p] for p in LEFT_EYE]))

            # Extract the bounding rectangle for the right eye
            right_eye_rect = cv.boundingRect(np.array([mesh_coords[p] for p in RIGHT_EYE]))

            # Crop the left eye region from the original image
            left_eye_image = image[left_eye_rect[1]:left_eye_rect[1] + left_eye_rect[3],
                             left_eye_rect[0]:left_eye_rect[0] + left_eye_rect[2]]

            # Crop the right eye region from the original image
            right_eye_image = image[right_eye_rect[1]:right_eye_rect[1] + right_eye_rect[3],
                              right_eye_rect[0]:right_eye_rect[0] + right_eye_rect[2]]

            # Save the left and right eye images
            cv.imwrite("left_eye.jpg", left_eye_image)
            cv.imwrite("right_eye.jpg", right_eye_image)

            # Draw bounding rectangles on the original image for visualization
            cv.rectangle(image, (left_eye_rect[0], left_eye_rect[1]), (left_eye_rect[0] + left_eye_rect[2], left_eye_rect[1] + left_eye_rect[3]), (0, 255, 0), 2)
            cv.rectangle(image, (right_eye_rect[0], right_eye_rect[1]), (right_eye_rect[0] + right_eye_rect[2], right_eye_rect[1] + right_eye_rect[3]), (0, 255, 0), 2)

        cv2_imshow(image)
        cv.waitKey(0)
        cv.destroyAllWindows()







In [10]:
!pip install tensorflow
from tensorflow.keras.models import load_model


In [11]:
model = load_model('my_model.h5')

In [24]:
for eyes in (left_eye_image,right_eye_image):
    # Resize the image to (224, 224) and preprocess
    final_img = cv.resize(eyes, (224, 224))
    final_img = np.expand_dims(final_img, axis=0)
    final_img = final_img / 255.0

    # Make the prediction
    prediction = model.predict(final_img)

    # Determine the result based on the prediction
    if prediction[0] > 0.5:
        result = "Open eye detected"
    else:
        result = "Closed eye detected"

    # Print or store the result as needed
    print(result)

1/1 [==============================] - 0s 34ms/step
Closed eye detected
1/1 [==============================] - 0s 72ms/step
Closed eye detected


In [25]:
cv2_imshow(image)